In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from utils import RawDataset, ImbalancedDatasetSampler
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm

SEQUENCE_LENGTH = 56 # Analysis file at `maximum input_size`
SENTIMENT_CLASSES = 5 
CLASSES = ['negative', 'somewhat neg', 'neutral', 'somewhat pos', 'positive']
EMBEDDING_SIZE = 19479 # len(my_dict.idx2word)

torch.manual_seed(1111111)
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

# Dataset

In [2]:
df = pd.read_csv('train.tsv', sep='\t')

In [3]:
df = df.drop(columns=['PhraseId', 'SentenceId'])

In [4]:
train, test = train_test_split(df, test_size=0.2)

In [5]:
len(train)/4, len(test)/4

(31212.0, 7803.0)

In [6]:
df.iloc[17]

Phrase       what is good for the goose
Sentiment                             2
Name: 17, dtype: object

In [6]:
train_dataset = RawDataset(train)
test_dataset = RawDataset(test)

### Found bug when using ImbalancedDatasetSampler
DataLoader will randomly return me arbitrary `batch_size`. Therefore I have to use `batch_size=1` only

In [7]:
%%time
train_loader = DataLoader(dataset=train_dataset, sampler=ImbalancedDatasetSampler(train_dataset), batch_size=4, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, sampler=ImbalancedDatasetSampler(test_dataset), batch_size=4, num_workers=2)

CPU times: user 1min 6s, sys: 90 ms, total: 1min 7s
Wall time: 1min 7s


with open('train_loader.pkl', 'rb') as my_input:
    train_loader = pickle.load(my_input)
with open('test_loader.pkl', 'rb') as my_input2:
    test_loader = pickle.load(my_input2)

import pickle

with open('train_loader.pkl', 'wb') as output:
    pickle.dump(train_loader, output, pickle.HIGHEST_PROTOCOL)
with open('test_loader.pkl', 'wb') as output:
    pickle.dump(test_loader, output, pickle.HIGHEST_PROTOCOL)

for i, (data, target) in enumerate(train_loader):
    if i == 0:
        import ipdb; ipdb.set_trace()
        print(data)
        print(target)
        break

# Utilities

In [9]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    print(cm)
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def train(model, device, train_loader, optimizer, epoch, criterion):
    """
    This function has one line different from the ordinary `train()` function
    It has `make_variables()` to convert tuple of names to be a tensor
    """
    model.train()
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        # For some reason `train_loader` encapsulates my data with tuple
        local_dummy = [i.lower().split(' ') for i in data]
        inputs = make_variables(local_dummy)

        data, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        tmp = output.view(-1, SENTIMENT_CLASSES)
        
        loss = criterion(tmp, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    y_test = []
    y_pred = []
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            local_dummy = [i.lower().split(' ') for i in data]
            inputs = make_variables(local_dummy)
            
            data, target = inputs.to(device), target.to(device)

            output = model(data)
            tmp = output.view(-1, SENTIMENT_CLASSES)

            test_loss += criterion(tmp, target).item() # sum up batch loss
            pred = tmp.max(1, keepdim=True)[1] # get the index of the max log-probability

            pred_tmp = pred.view(-1)
            pred_list = pred_tmp.tolist()
            target_list = target.tolist()
            
            y_test += target_list
            y_pred += pred_list
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # Confusion matrix
    confusion_mtx = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(confusion_mtx, classes=CLASSES, normalize=True,
                          title='Confusion matrix')


# 1. Model

In [10]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size=56, hidden_size=256, output_size=5, n_layers=1, num_embedding=19500, embedding_dim=300):
        """
        Because word embedding is working with ascii. It has to use `input_size=256, hidden_size=256`
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        # input_size 256, hidden_size 256.
        self.embedding = nn.Embedding(num_embedding, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input):
        """
        Do not remove `print`. Leave it be a historical footprint for I myself in the future
        """
        # Sung Kim run this all at once (over the whole input sequence)
        # input = B x S . size(0) = B
        batch_size = input.size(0)
        
        # input: B x S -- (transpose) --> S x B
        input = input.t()
        
        # Embedding S x B -> S x B x I (embedding size)
        # print(f" input size: {input.size()}")
        embedded = self.embedding(input)
        embedded = embedded.clone().detach() # Make new tensor because of `EmbeddingGrad`
        # print(f" embeddding size: {embedded.size()}")
        
        # Make a hidden
        hidden = self._init_hidden(batch_size)
        output, hidden = self.gru(embedded, hidden)
        # print(f" gru hidden output: {hidden.size()}")
        
        # Use last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden)
        # print(f" fc output: {fc_output.size()}")
        return fc_output
        
    def _init_hidden(self, batch_size):
        USE_CUDA = torch.cuda.is_available()
        DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return hidden.clone().detach().to(DEVICE)

In [11]:
import pickle
from utils import Dictionary

with open('dictioanry_data.pkl', 'rb') as input:
    my_dict = pickle.load(input)

# Example of `inputs`
data is a `batch` from `DataLoader`. And `inputs` is a tensor of dictionary index.<br>
After that `model` supposed to handle `indexes` by embedded layer and calculate on its own

In [12]:
def str2word_idx_arr(tokenized_sentence):
    """
    string to word-index and array
    """
    arr = [my_dict.word2idx[word] for word in tokenized_sentence]
    return arr, len(arr)

In [13]:
def pad_sequences(vectorized_seqs, seq_lengths):
    """
    Let the `SEQUENCE_LENGTH` is 19. According to the dataset
    """
    seq_tensor = torch.zeros((len(vectorized_seqs), SEQUENCE_LENGTH), dtype=torch.long)
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.tensor(seq, dtype=torch.long)
    return seq_tensor

In [14]:
def make_variables(sentence):
    sequence_and_length = [str2word_idx_arr(word) for word in sentence]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.tensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths)

make_variables(['i my cat'.split(' '), 'what is good for the goose'.split(' ')])

tensor([[  41, 1828, 2336,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   8,    9,   10,   11,    5,   12,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])

In [15]:
model = RNNClassifier(input_size=56).to(DEVICE)

In [16]:
# batch_size = 4
from pprint import pprint
data = (
    'with melancholy richness',
    'looks more like Danny Aiello',
    'circumstances',
    'the dead',
)
local_dummy = [i.lower().split(' ') for i in data]
inputs = make_variables(local_dummy)
pprint(local_dummy)
pprint(inputs)

[['with', 'melancholy', 'richness'],
 ['looks', 'more', 'like', 'danny', 'aiello'],
 ['circumstances'],
 ['the', 'dead']]
tensor([[   94,  3131,  4113,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [ 1935,   216,    98, 10820, 15578,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,

In [17]:
model(inputs.to(DEVICE))

tensor([[[ 0.1084,  0.1726, -0.1765, -0.2090,  0.0809],
         [ 0.1084,  0.1725, -0.1764, -0.2092,  0.0809],
         [ 0.1084,  0.1727, -0.1766, -0.2090,  0.0809],
         [ 0.1084,  0.1726, -0.1766, -0.2090,  0.0809]]],
       grad_fn=<AddBackward0>)

# 2. Criterion & Optimizer

In [20]:
input_size = len(my_dict.idx2word) #19479
input_size

19479

In [21]:
model = RNNClassifier(input_size=input_size).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
for epoch in range(1, 1 + 1):
    train(model, DEVICE, train_loader, optimizer, epoch, criterion)
    test(model, DEVICE, test_loader, criterion)

Train Epoch: 1 [0/124848 (0%)]	Loss: 1.819080
Train Epoch: 1 [4000/124848 (3%)]	Loss: 1.654910
Train Epoch: 1 [8000/124848 (6%)]	Loss: 1.580096
Train Epoch: 1 [12000/124848 (10%)]	Loss: 1.527628
Train Epoch: 1 [16000/124848 (13%)]	Loss: 1.144054
Train Epoch: 1 [20000/124848 (16%)]	Loss: 1.414001
Train Epoch: 1 [24000/124848 (19%)]	Loss: 1.431541
Train Epoch: 1 [28000/124848 (22%)]	Loss: 1.437923
Train Epoch: 1 [32000/124848 (26%)]	Loss: 1.853313
Train Epoch: 1 [36000/124848 (29%)]	Loss: 1.022406
Train Epoch: 1 [40000/124848 (32%)]	Loss: 0.997144
Train Epoch: 1 [44000/124848 (35%)]	Loss: 1.366148
Train Epoch: 1 [48000/124848 (38%)]	Loss: 1.423986
Train Epoch: 1 [52000/124848 (42%)]	Loss: 1.003134
Train Epoch: 1 [56000/124848 (45%)]	Loss: 0.994516
Train Epoch: 1 [60000/124848 (48%)]	Loss: 1.093737
Train Epoch: 1 [64000/124848 (51%)]	Loss: 1.319923
Train Epoch: 1 [68000/124848 (54%)]	Loss: 1.581661
Train Epoch: 1 [72000/124848 (58%)]	Loss: 1.124301
Train Epoch: 1 [76000/124848 (61%)]	Loss

# Save pickle

In [ ]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
save_object(model, 'sentiment-classifier.pkl')

In [2]:
import torch
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)

In [3]:
a

[tensor([1, 2, 3]), tensor([3, 4])]

In [4]:
b

tensor([[1, 2, 3],
        [3, 4, 0]])

In [7]:
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])


PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]))